# <center> Econometria Financiera <center>
# <center> Tarea 1 <center>
<center> The Harvard Management Company and Inflation-indexed Bonds <center>
<center> Ketherine Jimenez 1106918 - Aura Gonzalez 1106920 - Eddily Sanchez 1096853<center>

### Packetes a Importar

In [130]:
import pandas as pd
import numpy as np

### Funciones

In [131]:
def compute_tangency(df_tilde, diagonalize_Sigma=False):

    """Compute tangency portfolio given a set of excess returns.



    Also, for convenience, this returns the associated vector of average

    returns and the variance-covariance matrix.



    Parameters

    ----------

    diagonalize_Sigma: bool

        When `True`, set the off diagonal elements of the variance-covariance

        matrix to zero.

    """

    Sigma = df_tilde.cov()

    # N is the number of assets

    N = Sigma.shape[0]
    Sigma_adj = Sigma.copy()

    if diagonalize_Sigma:

        Sigma_adj.loc[:,:] = np.diag(np.diag(Sigma_adj))



    mu_tilde = df_tilde.mean()

    Sigma_inv = np.linalg.inv(Sigma_adj)

    weights = Sigma_inv @ mu_tilde / (np.ones(N) @ Sigma_inv @ mu_tilde)

    # For convenience, I'll wrap the solution back into a pandas.Series object.

    omega_tangency = pd.Series(weights, index=mu_tilde.index)

    return omega_tangency, mu_tilde, Sigma

In [132]:
def target_mv_portfolio(df_tilde, target_return=0.06, diagonalize_Sigma=False):

    """Compute MV optimal portfolio, given target return and set of excess returns.

    Parameters

    ----------

    diagonalize_Sigma: bool

        When `True`, set the off diagonal elements of the variance-covariance

        matrix to zero.

    """

    omega_tangency, mu_tilde, Sigma = compute_tangency(df_tilde, diagonalize_Sigma=diagonalize_Sigma)

    Sigma_adj = Sigma.copy()

    if diagonalize_Sigma:

        Sigma_adj.loc[:,:] = np.diag(np.diag(Sigma_adj))

    Sigma_inv = np.linalg.inv(Sigma_adj)

    N = Sigma_adj.shape[0]

    delta_tilde = ((np.ones(N) @ Sigma_inv @ mu_tilde)/(mu_tilde @ Sigma_inv @ mu_tilde)) * target_return

    omega_star = delta_tilde * omega_tangency

    return omega_star

### Cargando Data

In [133]:

path="/Users/Aura/Desktop/Data/multi_asset_etf_data.xlsx"
df1 = df = pd.read_excel(path, sheet_name=1)
df1 = df1.set_index('Date')

#df1 corresponde a Total Returns


#### 1. El enfoque de HMC.

El planteamiento de HMC sobre el problema de conformación de portafolio.

#### a) ¿Por qué HMC se enfoca en retornos reales cuando analiza la distribución de su portafolio? ¿Esto es solo tema de escala/unidades, o el utilizar retornos reales vs retornos nominales potencialmente cambia la solución al problema MV?


HMC se utiliza para  analizar los retornos esperados, las varianzas y covarianza de los retornos reales en los mercados de capitales, el mismo se basa en datos historicos. Esto no es identico a la optimizacion MV usando retornos nominales. Cambiar de retorno nominal a real simplemente reescala de manera distinta el promedio de exceso de retornos, pero puede realizar cambios en la matriz de covarianza los cuales pueden ser no triviales. Esto resultaría en ponderaciones diferentes sobre los activos.

El analisis-varinza podria verse afectado por la eleccion de utilizar retornos nominales en lugar de retornos reales, ya que estados dos variables no ofrecen la misma informacion (una calcula la rentabilidad real percibida tras la deduccion fiscal y el ajuste por inflacion y la otra solo tiene en cuenta las ganancias percibidas). Ademas HMC, basa constantemente sus estrategias en el retorno real.

#### b) Hay miles de activos riesgosos en los que HMC puede invertir. Explique por qué la optimización usando 1000 instrumentos financieros no es tan factible.


Mientras mas grande es el conjutno de instrumentos financieros la correlacion entre los activos puede verse afectada de amnera negativa y los elemntos de la matriz de covarianca pueden tener insuficiente precision.


#### c) En vez de optimizar a través de todos los instrumentos directamente, HMC realiza una optimización en dos etapas. Primero, ellos construyen portafolios por tipo activos, cada uno optimizado sobre instrumentos de la clase especifica de activos. Segundo, HMC combina los portafolios por tipo de activos en un solo portafolio optimizado.

Para que la optimización en dos etapas sea una buena aproximación a la optimización tradicional sobre todos los tipos de activos, ¿que debe ser cierto sobre la partición de los activos dentro del tipo de clase?


En la particion los activos deben tener una correlacion baja o ninguna entre las clases para que la optimizacion en dos etapas sea una buena aproximacion a la optimizacion tradicional sobre los tipos de activos.

#### d) Deberían los TIPS formar una nueva clase de activos financieros o ser agrupado dentro de una de las otras 11 clases.


Este al ser un tipo de activo indexado a la inflaccion, puede ajustarse segun la misma, de esta manera estan protegidos contra la inflaccion, los mismos ofrecen un rendimiento mas bajo en comparacion con otras inversiones similares, seria recomendable clasificarse en una nueva clase de activos. Se podrian incluir dentro de la clase de Inflation-Indexed donde ya esta Salomon o en la de Domestic Bonds ya ue es un US Bond.

#### e) ¿Cómo podríamos ajustar el problema de optimización de portafolio mostrado en la primera clase para reflejar las restricciones adicionales que Harvard está usando en sus soluciones acotadas en el Exhibit 5 y 6? Solo considere como reescribir la optimización: no trate de resolver el problema utilizando la restricción extra.

Ajustando la volatividad, la cantidad de activos y la covarianza entre los retornos de los activos con las restricciones de Harvard (0-10%)

### 2. Optimización Mean Varianza.

•La data de los exhibit se les compartirá, pero no necesita usarla.

• Para nuestro análisis utilizaremos la data en el archivo multi_asset_etf_data.xlsx. 

• La serie de tiempo está compuesta por retornos mensuales de 11 tipos de activos financieros y un fondo de corto plazo de notas del tesoro de EE. UU., (“SHV”) el cual consideramos la tasa libre de riesgo. 

• La data provista está en retornos totales, así como también en exceso de retornos, (donde SHV ha sido sustraído de las otras columnas)

• Estos rendimientos son nominales, es decir, no están ajustados por inflación, y en nuestros cálculos no estaremos haciendo ajustes por inflación.

En las preguntas debajo, anualice las estadísticas que piensa reportar:

Procedimiento:


Anualizacion de la media de los retornos mensuales multiplicando por 12:

In [134]:
#Media del total de los retornos mensuales (Anualizado)
# Asumismos que la tasa libre de riesgo es 0.01/12 en los meses donde no hay data en Cash 

df1['rf'] = df1['SHV'].fillna(0.01/12)

rf_mean_annualized = df1['rf'].mean() * 12

force = df1.drop(columns = ['SHV'])

risky_assets = force.columns[0:11]

risky_assets = risky_assets

df1_tilde = df1.loc[:, risky_assets]

df1_tilde = df1_tilde.subtract(df1['rf'], axis=0)



Anualizacion de la volatibilidad de los retornos mensuales multiplicado por raiz de 12.

El Sharpe Ratio es el promedio del exceso de retornos dividido por la volatilidad de los excesos de retorno.

donde 𝑅a es el retorno del activo fin𝑎𝑎anciero, 𝑅𝑏 es la tasa libre de riesgo. $𝐸[𝑅𝑎 − 𝑅𝑏]$ es el valor esperado del exceso de retorno del activo financiero sobre la tasa de referencia (en este caso la tasa libre de riesgo), y 𝜎 es la desviación estándar de los excesos de retorno del activo.

### 1. Resumen Estadístico

#### a.	Calcule y muestre la media y la volatilidad de cada uno del exceso de retorno de cada activo. (Recuerde que utilizamos volatilidad para referirnos a la desviación estándar de los retornos).

In [135]:
df1_tilde_annual = 12 * df1_tilde

mu_tilde = df1_tilde_annual.mean()

sigma = df1_tilde_annual.std()/np.sqrt(12)

table1 = pd.DataFrame({'Mean':mu_tilde, 'Vol':sigma, 'Sharpe':mu_tilde/sigma})

table1


,Mean,Vol,Sharpe
BWX,0.020819,0.074304,0.280189
DBC,0.010053,0.177327,0.056689
EEM,0.093103,0.196190,0.474556
EFA,0.099239,0.161319,0.615174
HYG,0.081193,0.085989,0.944231
IEF,0.033424,0.056895,0.587473
IYR,0.164311,0.180085,0.912408
PSP,0.170243,0.213412,0.797721
QAI,0.027795,0.047800,0.581490
SPY,0.164907,0.139166,1.184963


#### b. ¿Cuáles activos tienen el mejor o el peor Sharpe Ratio?

Se suele considerar que un buen Sharpe ratio está se muestra por encima de 1, porque ofrece más rentabilidad por unidad de volatilidad, entre nuestros activos el mejor Sharpe ratio lo tiene SPY con un valor correspondiente a 1.184963, y el peor lo tiene DBC con 0.056689.

### 2. Análisis Descriptivo.

#### a. Calcule la matriz de correlación de los excesos de retornos. ¿Qué par de activos tiene la correlación más alta? ¿La más baja?


In [137]:
corr = df1_tilde_annual.corr ()
corr.style.background_gradient (cmap = 'coolwarm')

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
BWX,1.000000,0.437619,0.608377,0.552190,0.479980,0.235399,0.397112,0.456009,0.605444,0.373806,0.526797
DBC,0.437619,1.000000,0.619991,0.634630,0.530205,-0.379698,0.321818,0.575219,0.617904,0.574377,0.137627
EEM,0.608377,0.619991,1.000000,0.854021,0.748341,-0.311420,0.614140,0.804353,0.805874,0.757758,0.221171
EFA,0.552190,0.634630,0.854021,1.000000,0.750992,-0.378969,0.666056,0.908241,0.830229,0.873131,0.138746
HYG,0.479980,0.530205,0.748341,0.750992,1.000000,-0.237206,0.757550,0.820091,0.745370,0.746024,0.208048
IEF,0.235399,-0.379698,-0.311420,-0.378969,-0.237206,1.000000,-0.079125,-0.404348,-0.161356,-0.419998,0.649038
IYR,0.397112,0.321818,0.614140,0.666056,0.757550,-0.079125,1.000000,0.734348,0.601285,0.716066,0.283457
PSP,0.456009,0.575219,0.804353,0.908241,0.820091,-0.404348,0.734348,1.000000,0.810628,0.894176,0.135455
QAI,0.605444,0.617904,0.805874,0.830229,0.745370,-0.161356,0.601285,0.810628,1.000000,0.820112,0.337546
SPY,0.373806,0.574377,0.757758,0.873131,0.746024,-0.419998,0.716066,0.894176,0.820112,1.000000,0.103265


Los activos que tienen la mayor correlacion son PSP-EFA con 0.908241, y por otro lado PSP-EFA con 0.894176. Luego podmeos observar que los activos con la correlacion mas baja TIP-SPY con un 0.10326 y TIP-PSP con 0.135455. Es importante destacar que las relaciones mas bajas entre los activos se encuentran entre los mismos y los TIPS.

#### b. ¿Qué tan bien los TIPS se muestran en términos de rendimiento y volatilidad en nuestra muestra? ¿Acoso los TIPS superan a los bonos domésticos en termino de resultados? ¿Y en comparación a los bonos externos?

Los TIPS tiene mejor promedio de rendimientos en comparacion a los otros activos, sin embargo, tiene la menor volativilidad y el 4to en exceso de retorno, por lo que podemos considerarlo una muy buena inversion. Los TIPS tienen menos volatibilidad, sim embargo los bonos domesticos tienen mayor exceso de retorno, eso depende del tipo de inversionista, si es arriesgado o no el inversionista, Por otro lado los TIPS no superan a los bonos externos.

#### c. Basado en la data, ¿Acaso los TIPs aparentan expandir el conjunto de las oportunidades de inversión, implicando que Harvard debería considerarlas como activo diferente?

Los TIPs expanden el conjunto de oportunidades de inversion, Harvard definitivamente debe considerarlas como activo diferido.

### 3. La Frontera MV

#### a.	Compute y muestre los pesos del portafolio tangente

In [138]:
omega_tangency, mu_tilde, Sigma = compute_tangency(df1_tilde_annual)

omega_tangency

BWX   -0.259236
DBC   -0.049786
EEM    0.068120
EFA   -0.157671
HYG    0.775635
IEF    1.734962
IYR   -0.366667
PSP   -0.000685
QAI   -2.477194
SPY    1.540749
TIP    0.191773
dtype: float64

#### b.	Compute la media, volatilidad y Sharpe Ratio del portafolio tangente

In [139]:
# Mean

mean = mu_tilde @ omega_tangency

# Volatlity

vol = np.sqrt(omega_tangency @ Sigma @ omega_tangency)/np.sqrt(12)


# Sharpe ratio

sharpe_ratio = mean/vol

print("Mean: ",mean,", vol: ",vol,", sharpe_ratio: ",sharpe_ratio)

Mean:  0.2377671633954937 , vol:  0.10480197847812671 , sharpe_ratio:  2.268727812663558


### 4.	La distribución de instrumentos.

a. Compute y muestre los pesos de los portafolios MV con un retorno  de  𝜇̃𝑝=0.01%


In [140]:
omega_star = target_mv_portfolio(df1_tilde_annual, target_return=0.01 * 12)

omega_star

BWX   -0.130835
DBC   -0.025127
EEM    0.034380
EFA   -0.079576
HYG    0.391459
IEF    0.875628
IYR   -0.185055
PSP   -0.000346
QAI   -1.250228
SPY    0.777609
TIP    0.096787
dtype: float64

b. ¿Cuál es la media, volatilidad, y Sharpe Ratio para  $\tilde{𝜇̃}^{p}$


In [141]:
# Mean

mean = mu_tilde @ omega_star


# Volatlity

vol = np.sqrt(omega_star @ Sigma @ omega_star)/np.sqrt(12)


# Sharpe ratio

sharpe_ratio = mean/vol

print("Mean: ",mean,", vol: ",vol,", sharpe_ratio: ",sharpe_ratio)

Mean:  0.12 , vol:  0.05289307925357346 , sharpe_ratio:  2.268727812663559


#### c. Discuta la distribución de activos del portafolio. ¿En qué activos está el portafolio más long? ¿En qué activos está más short?

El portafolio mas long se encuentra en los activos:
EEM - HYG - IEF -SPY - TIP
El portafolio mas short se encuentra en los activos:
BWX - DBC - EFA - IYR - PSP - QAI

#### d. ¿Estos resultados se alinean con que activos tiene los Sharpe Ratios más altos?

Si, debido a que el Sharpe Ratio del portafolio es 2.2687278.

### 5. Portafolios simples.

#### a. Calcule el rendimiento de portafolio con ponderadores iguales (equally weighted portfolio) sobre la muestra. 

$\tilde{w}^{i} = 1/n $

#### Reescale el vector de ponderadores para tener un media objetivo de $\tilde{𝜇̃}^{p} = .01$. Reporte la media, volatilidad y Sharpe Ratio

In [155]:
def pesosPortafolio(dataframe):
    array = []
    for i in dataframe.columns:
        array.append(1/len(df.columns))
        arrayFinal = np.array(array)
        return arrayFinal
    pesos = pesosPortafolio(df)
    p = pesos*0.01/pesos.mean()
    
#Mean
mean = mu_tilde @ p[0:11]

#Volatlity
vol = np.sqrt(p[0:11] @ Sigma @ p[0:11])/np.sqrt(12)

#Sharpe ratio
sharpe_ratio = mean/vol

print("Mean: ",mean,", vol: ",vol,", sharpe_ratio: ",sharpe_ratio)


Mean:  0.002425238565363779 , vol:  0.002603809379942794 , sharpe_ratio:  0.931419398073242


#### b. Calcule el rendimiento de portafolio “Paridad de Riesgo” (Risk-Parity Portfolio) sobre la muestra. Risk-Parity es un término usado en diferentes casos en el mundo de las finanzas, pero en este caso lo que nos interesa es que los ponderadores o pesos de los activos dentro del portafolio sean proporcionales a la inversa de la volatilidad estimada de cada activo.

 $\tilde{w}^{i} = 1/n $

#### Esto resultara en el vector de ponderadores w, pero necesitara reescalar el vector para tener media $\tilde{𝜇̃}^{p} = .01$

In [157]:

# Calcular la matriz de covarianza de los retornos de los activos
covariance = force.cov()

# Calcular la volatilidad estimada de cada activo
volatility = np.sqrt(np.diag(covariance))

# Calcular los pesos de los activos utilizando la inversa de la volatilidad estimada
weights = 1 / volatility
weights /= np.sum(weights)
p = weights*0.01/weights.mean()

# Calcular el rendimiento del portafolio
portfolio_returns = np.dot(force, p)

#Imprimir el rendimiento del portafolio
print('El rendimiento del portafolio Risk-Parity es igual a:', round(np.sum(portfolio_returns) * 100, 2), '%')



El rendimiento del portafolio Risk-Parity es igual a: 2.93 %


#### c.¿Cómo se compara estos resultados al portafolio MV del problema 2.4?

El rendimiento del portafolio Risk-Parity da como resultado un mayor Sharpe Ratio que el portafolio MV, lo que quiere decir que al  los ponderadores o pesos de los activos dentro del portafolio ser proporcionales a la inversa de la volatilidad estimada de cada activo el portafolio tiene myor rentabilidad por unidad de volatilidad.

### 6.	Rendimiento Out-of-sample

Vamos a dividir la muestra en para computar un portafolio y luego ver su rendimiento fuera de muestra.

#### a. Usando solo la data a hasta 2020, compute  $\tilde{w}^{p}$ para $\tilde{\mu}^{p} = 0.01\%$, utilizando las 11 clase de activos financieros dados.

In [144]:
df1_temp = df1_tilde_annual.loc[:'2020', :]

omega_tangency, mu_tilde, Sigma = compute_tangency(df1_temp)

omega_star = target_mv_portfolio(df1_temp, target_return=0.01*12)

omega_star

BWX   -0.039015
DBC   -0.070831
EEM    0.027148
EFA   -0.076195
HYG    0.461413
IEF    0.844823
IYR   -0.209008
PSP   -0.024835
QAI   -1.197772
SPY    0.785429
TIP    0.056842
dtype: float64

#### b. Usando eso mismos ponderadores $\tilde{w}^{p}$, calcule el Sharpe Ratio, dentro de la muestra, hasta finales del 2020. 

In [145]:
mean = omega_star @ mu_tilde

vol = np.sqrt(omega_star @ Sigma @ omega_star)/np.sqrt(12)

sharpe_ratio_in_sample = mean/vol

sharpe_ratio_in_sample

print("Mean: ",mean,", vol: ",vol,", sharpe_ratio: ",sharpe_ratio_in_sample)

Mean:  0.12 , vol:  0.052405114979212526 , sharpe_ratio:  2.2898528139400183


#### c. Usando los mismo ponderadores $\tilde{w}^{p}$ (usando la data hasta finales del 2020), calcule el Sharpe Ratio basado en el año 2021.

In [146]:
df1_temp = df1_tilde_annual.loc['2020':, :]

omega_tangency, mu_tilde, Sigma = compute_tangency(df1_temp)

mean = omega_star @ mu_tilde

vol = np.sqrt(omega_star @ Sigma @ omega_star)/np.sqrt(12)

sharpe_ratio_out_of_sample = mean/vol

sharpe_ratio_out_of_sample

print("Mean: ",mean,", vol: ",vol,", sharpe_ratio: ",sharpe_ratio_out_of_sample)

Mean:  0.15373102170980696 , vol:  0.07836756060512039 , sharpe_ratio:  1.9616665431814202
